### Brain-hacking 101

Author: [**Ariel Rokem**](http://arokem.org), [**The University of Washington eScience Institute**](http://escience.washington.edu)

### Hack 1: Read your data into an array

When you conduct a neuroimaging experiment, the computer that controls the scanner and receives the data from the scanner saves your data to a file. Neuroimaging data appears in many different file formats: `NiFTI`, `Minc`, `Dicom`, etc. These files all contain representations of the data that you collected in the form of an **array**. 

What is an array? It is a way of representing the data in the computer memory as a *table*, that is *multi-dimensional* and *homogenous*.

What does this mean? 

- *table* means that you will be able to read all or some of the numbers representing your data by addressing the variable that holds your array. It's like addressing a member of your lab to tell you the answer to a question you have, except here you are going to 'ask' a variable in your computer memory. Arrays are usually not as smart as your lab members, but they have very good memory.

- *multi-dimensional* means that you can represent different aspects of your data along different axes. For example, the three dimensions of space can be represented in different dimensions of the table:

![Arrays](./images/array.svg)

- *homogenous* actually means two different things: 
    - The shape of the array is homogenous, so if there are three items in the first column, there have to be three items in all the columns. 
    - The data-type is homogenous. If the first item is an integer, all the other items will be integers as well.

To demonstrate the properties of arrays, we will use the [`numpy`](https://numpy.org) library. This library contains implementations of many scientifically useful functions and objects. In particular, it contains an implementation of arrays that we will use throughout the folllowing examples.

In [1]:
import numpy as np

In [2]:
# Numpy is a package. To see what's in a package, type the name, a period, then hit tab
#np?
#np.

In [3]:
# Some examples of numpy functions and "things":
print(np.sqrt(4))
print(np.pi)  # Not a function, just a variable
print(np.sin(np.pi)) # A function on a variable :) 

2.0
3.14159265359
1.22464679915e-16


### Numpy arrays (ndarrays)

Creating a NumPy array is as simple as passing a sequence to `np.array` 

In [4]:
arr1 = np.array([1, 2.3, 4])   
print(type(arr1))
print(arr1.dtype)
print(arr1.shape)

<type 'numpy.ndarray'>
float64
(3,)


In [5]:
print(arr1)

[ 1.   2.3  4. ]


### You can create arrays with special generating functions: 

`np.arange(start, stop, [step])`

`np.zeros(shape)`

`np.ones(shape)`

In [6]:
arr4 = np.arange(2, 5)
print(arr4)
arr5 = np.arange(1, 5, 2)
print(arr5)
arr6 = np.arange(1, 10, 2)
print(arr6)

[2 3 4]
[1 3]
[1 3 5 7 9]


## Exercise : Create an Array

Create an array with values ranging from 0 to 10, in increments of 0.5.

Reminder: get help by typing np.arange?, np.ndarray?, np.array?, etc.

### Arithmetic with arrays

Since numpy exists to perform efficient numerical operations in Python, arrays have all the usual arithmetic operations available to them. These operations are performed element-wise (i.e. the same operation is performed independently on each element of the array).

In [85]:
A = np.arange(5)
B = np.arange(5, 10)

print (A+B)

print(B-A)

print(A*B)

[ 5  7  9 11 13]
[5 5 5 5 5]
[ 0  6 14 24 36]


### What would happen if A and B did not have the same `shape`?

### Arithmetic with scalars:

In addition, if one of the arguments is a scalar, that value will be applied to all the elements of the array.

In [86]:
A = np.arange(5)
print(A+10)
print(2*A)
print(A**2)

[10 11 12 13 14]
[0 2 4 6 8]
[ 0  1  4  9 16]


### Arrays are addressed through indexing

**Python uses zero-based indexing**: The first item in the array is item `0`

The second item is item `1`, the third is item `2`, etc.

In [87]:
print(A)
print(A[0])
print(A[1])
print(A[2])

[0 1 2 3 4]
0
1
2


### `numpy` contains various functions for calculations on arrays

In [88]:
# This gets the exponent, element-wise:
print(np.exp(A))

# This is the average number in the entire array:
print(np.mean(A))

[  1.           2.71828183   7.3890561   20.08553692  54.59815003]
2.0


### Data in Nifti files is stored as an array 

In the tutorial directory, we have included a single run of an fMRI experiment that was included in the FIAC competition. The experiment is described in full in a paper by Dehaene-Lambertz et al. (2006), but for the purposes of what we do today, the exact details of the acquisition and the task are not particularly important.

We can read out this array into the computer memory using the `nibabel` library

In [89]:
import nibabel as nib

Loading the file is simple:

In [90]:
img = nib.load('./data/run1.nii.gz')

But note that in order to save time and memory, nibabel is pretty lazy about reading data from file, until we really need this data. 

Meaning that at this point, we've only read information *about* the data, not the data itself. This thing is not the data array yet. What is it then? 

In [91]:
type(img)

nibabel.nifti1.Nifti1Image

It's a `Nifti1Image` object! That means that it is a variable that holds various attributes of the data. For example, the 4 by 4 matrix that describes the spatial transformation between the world coordinates and the image coordinates

In [92]:
img.affine

array([[-3.00000238,  0.        ,  0.        ,  0.        ],
       [ 0.        , -3.00000381,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  4.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

This object also has functions. You can get the data, by calling a function of that object:

There's a header in there that provides some additional information:


In [93]:
hdr = img.get_header()
print(hdr.get_zooms())

(3.0000024, 3.0000038, 4.0, 1.0)


In [94]:
data = img.get_data()
print(type(data))

<type 'numpy.ndarray'>


In [95]:
print(data.shape)

(64, 64, 30, 191)


This is a 4-dimensional array! We happen to know that time is the last dimension, and there are 191 TRs recorded in this data. There are 30 slices in each TR/volume, with an inplane matrix of 64 by 64 in each slice. 

We can easily access different parts of the data. Here is the full time-series for the central voxel in the volume:

In [96]:
center_voxel_time_series = data[32, 32, 15, :]
print(center_voxel_time_series)
print(center_voxel_time_series.shape)

[ 3881.  3886.  4052.  3995.  3963.  3910.  3908.  3899.  3922.  3854.
  3903.  3812.  4041.  3896.  4037.  3857.  3981.  3901.  3931.  3867.
  3999.  4002.  3948.  4091.  3915.  3929.  3888.  3925.  3955.  3854.
  4023.  3861.  4011.  3864.  3736.  3961.  4019.  3790.  4041.  3941.
  4005.  3952.  3924.  3952.  3942.  3895.  3907.  4001.  3892.  4000.
  3835.  3823.  3922.  3792.  4021.  3841.  3972.  3804.  3837.  3851.
  3922.  3996.  3899.  3816.  3793.  3703.  3995.  3848.  3917.  3790.
  3828.  3978.  3833.  3726.  3950.  3868.  3812.  3910.  3882.  3847.
  3829.  3882.  3804.  3735.  3969.  3778.  3702.  3788.  3851.  3921.
  3769.  3836.  3794.  3884.  4060.  3883.  3810.  3785.  3995.  3926.
  3775.  3834.  3883.  3909.  3850.  3896.  3844.  3716.  3879.  4015.
  3903.  3893.  3760.  3931.  3876.  3986.  3809.  3835.  3949.  3874.
  3772.  3896.  3875.  3706.  3838.  3839.  3882.  4007.  3803.  3854.
  3795.  3780.  3793.  3728.  3783.  3828.  3742.  3760.  3672.  3913.
  3649

It's a one-dimensinal array! Here is the middle slice for the last time-point

In [97]:
middle_slice_t0 = data[:, :, 15, -1]  # Using negative numbers allows you to count *from the end*
print(middle_slice_t0)
print(middle_slice_t0.shape)

[[ 19.  54.  59. ...,  44.  52.  57.]
 [ 27.  32.  12. ...,  19.  17.  65.]
 [ 28.  46.  85. ...,  69.  58.  74.]
 ..., 
 [ 25.  63.   5. ...,  66.  31.  56.]
 [ 47.  69.  35. ...,  70.  72.  40.]
 [ 28.  38.  33. ...,  14.  53.  16.]]
(64, 64)


That's a 2D array. You get the picture, I hope.

You can do all kinds of operations with the data using functions:

In [98]:
print(np.mean(center_voxel_time_series))
print(np.std(center_voxel_time_series))
# TSNR is mean/std:
print(np.mean(center_voxel_time_series)/np.std(center_voxel_time_series))

3839.65968586
110.300135033
34.8110152785


### Using functions on parts of the data

Many `numpy` functions have an `axis` optional argument. These arguments allow you to perform a reduction of the data along one of the dimensions of the array.

For example, if you want to extract a 3D array with the mean/std in every one of the voxels:

In [99]:
mean_tseries = np.mean(data, axis=-1) # Select the last dimension
std_tseries = np.std(data, axis=-1)
tsnr = mean_tseries/std_tseries

In [100]:
print(tsnr.shape)

(64, 64, 30)


You can save the resulting array into a new file:

In [101]:
new_img = nib.Nifti1Image(tsnr, img.affine)

In [102]:
new_img.to_filename('tsnr.nii.gz')